# Module 03 - Spark SQL and DataFrames in Databricks

## Overview

This module focuses on Spark SQL and advanced DataFrame operations in Databricks. Since you already know PySpark, we'll focus on Databricks-specific features and optimizations.

## Learning Objectives

By the end of this module, you will understand:
- Spark SQL in Databricks environment
- Working with temporary views and global views
- Advanced DataFrame operations
- Window functions and aggregations
- Performance optimization techniques
- Integration between SQL and Python cells


## Creating Sample Datasets

Let's create comprehensive sample datasets for our demonstrations.


In [0]:
# Create sample datasets
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import col, lit, rand, when
from datetime import datetime, timedelta

# Employees dataset
employees_data = [
    (1, "Alice", "Engineering", 75000, "2020-01-15"),
    (2, "Bob", "Sales", 65000, "2019-03-20"),
    (3, "Charlie", "Engineering", 80000, "2018-06-10"),
    (4, "Diana", "Marketing", 60000, "2021-02-05"),
    (5, "Eve", "Sales", 70000, "2020-11-12"),
    (6, "Frank", "Engineering", 85000, "2017-09-01"),
    (7, "Grace", "HR", 55000, "2022-01-10"),
    (8, "Henry", "Engineering", 90000, "2016-04-15"),
]

employees_schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("hire_date", StringType(), True)
])

employees_df = spark.createDataFrame(employees_data, employees_schema)
employees_df.createOrReplaceTempView("employees")

print("Employees DataFrame:")
employees_df.show()

# Sales dataset
sales_data = [
    ("2024-01-01", "Product A", 100.0, 10, 1),
    ("2024-01-02", "Product B", 150.0, 15, 2),
    ("2024-01-03", "Product A", 120.0, 12, 1),
    ("2024-01-04", "Product C", 200.0, 20, 3),
    ("2024-01-05", "Product B", 180.0, 18, 2),
    ("2024-01-06", "Product A", 110.0, 11, 1),
    ("2024-01-07", "Product C", 220.0, 22, 3),
    ("2024-01-08", "Product B", 160.0, 16, 2),
]

sales_schema = StructType([
    StructField("sale_date", StringType(), True),
    StructField("product", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("employee_id", IntegerType(), True)
])

sales_df = spark.createDataFrame(sales_data, sales_schema)
sales_df.createOrReplaceTempView("sales")

print("\nSales DataFrame:")
sales_df.show()


## Working with Temporary Views

Temporary views allow you to share DataFrames between Python and SQL cells. They exist only for the current Spark session.


In [0]:
# Create a temporary view from DataFrame
employees_df.createOrReplaceTempView("employees_view")

# Now you can query it in SQL cells
print("Temporary view 'employees_view' created")
print("You can now use it in SQL cells with: SELECT * FROM employees_view")


In [0]:
%sql
-- Query the temporary view using SQL
SELECT
    department,
    COUNT(*) as employee_count,
    AVG(salary) as avg_salary,
    MAX(salary) as max_salary,
    MIN(salary) as min_salary
FROM employees_view
GROUP BY department
ORDER BY avg_salary DESC


## Global Temporary Views

Global temporary views are accessible across all Spark sessions in the same cluster. They are stored in the `global_temp` database.


In [0]:
# Create a global temporary view
employees_df.createOrReplaceGlobalTempView("global_employees")

print("Global temporary view 'global_employees' created")
print("Access it in SQL with: SELECT * FROM global_temp.global_employees")


In [0]:
%sql
-- Query global temporary view
SELECT * FROM global_temp.global_employees
LIMIT 5


## Advanced SQL Queries

Let's explore advanced SQL features in Databricks.


In [0]:
%sql
-- JOIN operations
SELECT
    e.name,
    e.department,
    e.salary,
    s.product,
    s.amount,
    s.sale_date
FROM employees e
INNER JOIN sales s ON e.employee_id = s.employee_id
ORDER BY s.amount DESC


In [0]:
%sql
-- LEFT JOIN with aggregation
SELECT
    e.department,
    COUNT(DISTINCT e.employee_id) as total_employees,
    COUNT(s.sale_date) as total_sales,
    COALESCE(SUM(s.amount), 0) as total_revenue,
    COALESCE(AVG(s.amount), 0) as avg_sale_amount
FROM employees e
LEFT JOIN sales s ON e.employee_id = s.employee_id
GROUP BY e.department
ORDER BY total_revenue DESC


## Window Functions

Window functions are powerful for analytical queries. They allow you to perform calculations across rows related to the current row.


In [0]:
%sql
-- Window functions: ROW_NUMBER, RANK, DENSE_RANK
SELECT
    name,
    department,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as row_num,
    RANK() OVER (PARTITION BY department ORDER BY salary DESC) as rank_salary,
    DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) as dense_rank_salary,
    LAG(salary, 1) OVER (PARTITION BY department ORDER BY salary DESC) as prev_salary,
    LEAD(salary, 1) OVER (PARTITION BY department ORDER BY salary DESC) as next_salary
FROM employees
ORDER BY department, salary DESC


In [0]:
%sql
-- Window functions: Running totals and averages
SELECT
    sale_date,
    product,
    amount,
    SUM(amount) OVER (ORDER BY sale_date) as running_total,
    AVG(amount) OVER (PARTITION BY product ORDER BY sale_date
                      ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as moving_avg_3days,
    SUM(amount) OVER (PARTITION BY product ORDER BY sale_date) as product_running_total
FROM sales
ORDER BY sale_date


## Advanced DataFrame Operations

Now let's explore advanced DataFrame operations in Python.


In [0]:
# Window functions in PySpark
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, lag, lead, sum as spark_sum, avg as spark_avg

# Define window specification
window_spec = Window.partitionBy("department").orderBy(col("salary").desc())

# Apply window functions
result_df = employees_df.withColumn(
    "row_num", row_number().over(window_spec)
).withColumn(
    "rank_salary", rank().over(window_spec)
).withColumn(
    "dense_rank_salary", dense_rank().over(window_spec)
).withColumn(
    "prev_salary", lag("salary", 1).over(window_spec)
).withColumn(
    "next_salary", lead("salary", 1).over(window_spec)
)

display(result_df)


In [0]:
# Complex aggregations
from pyspark.sql.functions import count, countDistinct, collect_list, collect_set

agg_result = employees_df.groupBy("department").agg(
    count("*").alias("total_employees"),
    countDistinct("employee_id").alias("unique_employees"),
    spark_avg("salary").alias("avg_salary"),
    spark_sum("salary").alias("total_salary"),
    collect_list("name").alias("employee_names"),
    collect_set("name").alias("unique_names")
)

display(agg_result)


In [0]:
# Joins in PySpark
joined_df = employees_df.join(
    sales_df,
    employees_df.employee_id == sales_df.employee_id,
    "inner"
).select(
    employees_df["name"],
    employees_df["department"],
    sales_df["product"],
    sales_df["amount"],
    sales_df["sale_date"]
)

display(joined_df)


In [0]:
# Pivot operations
from pyspark.sql.functions import first

pivot_df = sales_df.groupBy("sale_date").pivot("product").agg(
    spark_sum("amount").alias("total_amount")
).na.fill(0)

display(pivot_df)


## User-Defined Functions (UDFs)

UDFs allow you to extend Spark SQL with custom functions. However, use them sparingly as they can impact performance.


In [0]:
# Register UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Simple UDF
def categorize_salary(salary):
    if salary >= 80000:
        return "High"
    elif salary >= 65000:
        return "Medium"
    else:
        return "Low"

salary_category_udf = udf(categorize_salary, StringType())

# Use UDF
result_df = employees_df.withColumn(
    "salary_category",
    salary_category_udf(col("salary"))
)

display(result_df)


In [0]:
# Register UDF for SQL use
spark.udf.register("categorize_salary", categorize_salary, StringType())

# Now use it in SQL
print("UDF registered. You can now use it in SQL cells:")
print("SELECT name, salary, categorize_salary(salary) as category FROM employees")


In [0]:
%sql
-- Use the registered UDF
SELECT
    name,
    salary,
    categorize_salary(salary) as salary_category
FROM employees
ORDER BY salary DESC


## Performance Optimization Techniques

### 1. Caching and Persistence


In [0]:
# Create a complex DataFrame that will be reused
complex_df = employees_df.join(
    sales_df,
    employees_df.employee_id == sales_df.employee_id,
    "inner"
).groupBy("department", "product").agg(
    spark_sum("amount").alias("total_amount"),
    spark_avg("amount").alias("avg_amount"),
    count("*").alias("transaction_count")
)

# Cache the DataFrame
complex_df.cache()

# First action - will compute and cache
print(f"Cached DataFrame count: {complex_df.count()}")

# Subsequent actions - will use cache
print("\nUsing cached DataFrame:")
display(complex_df.filter(col("total_amount") > 150))

# Unpersist when done
complex_df.unpersist()
print("\nDataFrame unpersisted")


### 2. Broadcast Joins

For small lookup tables, use broadcast joins to improve performance.


In [0]:
from pyspark.sql.functions import broadcast

# Create a small lookup table
departments_df = spark.createDataFrame([
    ("Engineering", "Tech"),
    ("Sales", "Business"),
    ("Marketing", "Business"),
    ("HR", "Support")
], ["department", "category"])

# Use broadcast join for small table
joined_with_broadcast = sales_df.join(
    broadcast(departments_df),
    sales_df.product == departments_df.department,
    "left"
)

display(joined_with_broadcast)


### 3. Partitioning and Coalescing


In [0]:
# Check current partitions
print(f"Number of partitions: {employees_df.rdd.getNumPartitions()}")

# Repartition
repartitioned_df = employees_df.repartition(4)
print(f"After repartition: {repartitioned_df.rdd.getNumPartitions()} partitions")

# Coalesce (reduces partitions)
coalesced_df = repartitioned_df.coalesce(2)
print(f"After coalesce: {coalesced_df.rdd.getNumPartitions()} partitions")

# Repartition by column (useful for joins)
repartitioned_by_dept = employees_df.repartition("department")
print(f"Repartitioned by department: {repartitioned_by_dept.rdd.getNumPartitions()} partitions")


## Converting Between SQL and DataFrames

One of Databricks' strengths is seamless integration between SQL and Python.


In [0]:
%sql
-- Create a view from SQL query
CREATE OR REPLACE TEMP VIEW department_summary AS
SELECT
    e.department,
    COUNT(DISTINCT e.employee_id) as employee_count,
    AVG(e.salary) as avg_salary,
    SUM(s.amount) as total_sales
FROM employees e
LEFT JOIN sales s ON e.employee_id = s.employee_id
GROUP BY e.department


In [0]:
# Use the SQL view in Python
dept_summary_df = spark.table("department_summary")
display(dept_summary_df)

# Now you can use DataFrame operations
filtered_df = dept_summary_df.filter(col("total_sales") > 0)
display(filtered_df)


In [0]:
# Execute SQL from Python and get result as DataFrame
sql_query = """
SELECT
    e.name,
    e.department,
    COUNT(s.sale_date) as sales_count,
    SUM(s.amount) as total_sales
FROM employees e
LEFT JOIN sales s ON e.employee_id = s.employee_id
GROUP BY e.name, e.department
ORDER BY total_sales DESC
"""

result_df = spark.sql(sql_query)
display(result_df)


## Best Practices

1. **Use SQL for complex queries** - Often more readable for complex joins and aggregations
2. **Use DataFrames for programmatic logic** - Better for conditional logic and loops
3. **Cache frequently used DataFrames** - But remember to unpersist when done
4. **Use broadcast joins for small tables** - Improves join performance
5. **Partition wisely** - Too many partitions can hurt performance
6. **Avoid UDFs when possible** - Use built-in functions for better performance
7. **Use temporary views** - Share data between SQL and Python cells
8. **Leverage display()** - Better visualization than show()


## Summary

In this module, you learned:

✅ **Temporary and Global Views** - Sharing data between SQL and Python

✅ **Advanced SQL** - JOINs, aggregations, and complex queries

✅ **Window Functions** - ROW_NUMBER, RANK, running totals, and moving averages

✅ **Advanced DataFrame Operations** - Window functions, pivots, and complex aggregations

✅ **UDFs** - Creating and using user-defined functions

✅ **Performance Optimization** - Caching, broadcast joins, and partitioning

✅ **SQL-DataFrame Integration** - Seamless switching between SQL and Python

### Next Steps

In the next module, we'll explore:
- Delta Lake: ACID transactions and time travel
- Unity Catalog: Data governance and cataloging
- Jobs: Scheduling and automation
- Advanced Databricks features


## Exercise

Try these exercises to practice:

1. Create a complex SQL query with multiple JOINs and window functions
2. Convert the SQL query result to a DataFrame and apply additional transformations
3. Create a UDF and use it in both Python and SQL
5. Use broadcast join for a small lookup table
7. Use window functions to calculate running totals and moving averages
